In [1]:
# import statements
import pandas as pd
import numpy as np

In [20]:
def clean_report(source_data,report,report_filename):
    '''Reads in billing report for month, and source data of division and department,
    then cleans it
    Input : 
    source_data(string) : data showing division and department
    report(string) : billing report from imagic
    report_filename(string) : name of cleaned output file
    
    Output :
    clean_report : cleaned billing month report for month
    
    '''
    # Read in billing report & source_data
    source_data = pd.read_excel(source_data,index_col=None)
    report = pd.read_excel(report, skiprows = 7,index_col=None)
    
    #remove unnamed columns in report
    report = report[report.columns[~report.columns.str.contains('Unnamed:')]]
    
    #list of unused columns
    unused_cols = ['%','Avg Cost','Cost_2','Difference','% Difference','%.1','Avg Duration']
    
    #drop  unused columns
    report.drop(columns= unused_cols,inplace=True)
    
    
    # Create New columns from Extension by striping extension from name
    report[['Number','Name']] = report['Extension'].str.extract(r'(\d+)\s*(.*)', expand=True)
    
    # drop Extension column
    report.drop(['Extension'],axis= 1,inplace=True)
    
    # rename Number to Extension
    report.rename(columns = {'Number':'Extension'}, inplace = True)
                       
    # fill null values in extension with zero
    report['Extension'] = report['Extension'].fillna(0)
                   
    # convert Extension column to appropriate datatype
    convert_dict = {'Extension': int} 
    report = report.astype(convert_dict) 
                   
    #Rename telephoneNumber as Extension to match report.
    source_data.rename(columns = {'telephoneNumber':'Extension'}, inplace = True)
                   
    # join reports df with the source_data df on the Extension
    merged_df = pd.merge(report, source_data, on ='Extension',how ='inner') 
                   
    # drop unused columns
    merged_df.drop(['Present Functional Name'],axis= 1,inplace=True)
                   
    # Change order of columns in dataframe
    report_clean = merged_df[['Extension','Name','Division','Department','Duration','Calls','Cost']]
                   
    # drop duplicate rows using extension
    report_clean = report_clean.drop_duplicates(subset='Extension', keep="first")
    
    # Output clean df
    report_clean.to_excel(report_filename + ".xlsx",sheet_name='main',index = False)
    
    # output null rows to dataframe with index
    null_values = report_clean[report_clean['Division'].isna()]
    null_values.to_excel(report_filename+ "_null_values.xlsx",sheet_name='null_values',index = False) 
                   

In [22]:
clean_report('source_data.xlsx','oct_2020_desk.xls','Tucn_oct_2020')

WARNING *** file size (1297352) not 512 + multiple of sector size (512)


In [3]:
# Read in billing report & source_data from local
source_data = pd.read_excel('source_data.xlsx',index_col=None)

oct_2020 = pd.read_excel('oct_2020_desk.xls', skiprows = 7,index_col=None)
oct_2020.head(5)


WARNING *** file size (1297352) not 512 + multiple of sector size (512)


,Unnamed: 0,Extension,Unnamed: 2,Unnamed: 3,Cost,%,Avg Cost,Cost_2,Difference,Unnamed: 9,Unnamed: 10,% Difference,Calls,Duration,%.1,Unnamed: 15,Unnamed: 16,Avg Duration
0,NaN,607738596 FSO COMON SUP FSO,NaN,NaN,1.212643e+06,0.115113,3975.879543,0,-1.212643e+06,NaN,NaN,NaN,305,18:55:00,0.000842,NaN,NaN,00:03:43
1,NaN,607738580 FSO HELI-LOUNGE FSO,NaN,NaN,1.858541e+05,0.017643,1272.973236,0,-1.858541e+05,NaN,NaN,NaN,146,05:57:39,0.000265,NaN,NaN,00:02:27
2,NaN,607737861 AKPO AKPO PUBLIC PHONE 2 LQ LEVEL 2,NaN,NaN,1.431314e+05,0.013587,46.456144,0,-1.431314e+05,NaN,NaN,NaN,3081,229:07:44,0.010195,NaN,NaN,00:04:28
3,NaN,607737860 AKPO AKPO PUBLIC PHONE 1 LQ LEVEL 2,NaN,NaN,1.254429e+05,0.011908,39.042298,0,-1.254429e+05,NaN,NaN,NaN,3213,234:19:46,0.010426,NaN,NaN,00:04:23
4,NaN,607738407 AMENAM INSTRUMENT SUPERVISOR,NaN,NaN,1.044787e+05,0.009918,477.071868,0,-1.044787e+05,NaN,NaN,NaN,219,24:45:14,0.001101,NaN,NaN,00:06:47


In [4]:
source_data.head(3)

,telephoneNumber,Present Functional Name,Division,Department
0,20000,NaN,NaN,NaN
1,45033,EGINA HSE STORE,EGINA,EGINA Project
2,45034,EGINA GYM,EGINA Project,EGINA


In [5]:
# view number of null columns
source_data.isnull().sum()

telephoneNumber              0
Present Functional Name    356
Division                   637
Department                 486
dtype: int64

In [6]:
# view number of null columns
oct_2020.isnull().sum()

Unnamed: 0      3135
Extension          0
Unnamed: 2      3135
Unnamed: 3      3135
Cost               0
%                  0
Avg Cost         701
Cost_2             0
Difference         0
Unnamed: 9      3135
Unnamed: 10     3135
% Difference    3135
Calls              0
Duration           0
%.1                0
Unnamed: 15     3135
Unnamed: 16     3135
Avg Duration       0
dtype: int64

**Data Cleaning**

In [7]:
#remove unnamed columns
oct_2020 = oct_2020[oct_2020.columns[~oct_2020.columns.str.contains('Unnamed:')]]
list(oct_2020.columns)

['Extension',
 'Cost',
 '%',
 'Avg Cost',
 'Cost_2',
 'Difference',
 '% Difference',
 'Calls',
 'Duration',
 '%.1',
 'Avg Duration']

In [8]:
# drop unused columns
#list of unused columns
unused_cols = ['%','Avg Cost','Cost_2','Difference','% Difference','%.1','Avg Duration']
#drop list of unused columns
oct_2020.drop(columns=unused_cols,inplace=True)

In [9]:
#confirm changes
list(oct_2020.columns)

['Extension', 'Cost', 'Calls', 'Duration']

In [10]:
oct_2020.head(3)

,Extension,Cost,Calls,Duration
0,607738596 FSO COMON SUP FSO,1.212643e+06,305,18:55:00
1,607738580 FSO HELI-LOUNGE FSO,1.858541e+05,146,05:57:39
2,607737861 AKPO AKPO PUBLIC PHONE 2 LQ LEVEL 2,1.431314e+05,3081,229:07:44


In [11]:
# Create New columns from Extension by striping extension from name
oct_2020[['Number','Name']] = oct_2020['Extension'].str.extract(r'(\d+)\s*(.*)', expand=True)

In [12]:
# Confirm changes
oct_2020.head(3)

,Extension,Cost,Calls,Duration,Number,Name
0,607738596 FSO COMON SUP FSO,1.212643e+06,305,18:55:00,607738596,FSO COMON SUP FSO
1,607738580 FSO HELI-LOUNGE FSO,1.858541e+05,146,05:57:39,607738580,FSO HELI-LOUNGE FSO
2,607737861 AKPO AKPO PUBLIC PHONE 2 LQ LEVEL 2,1.431314e+05,3081,229:07:44,607737861,AKPO AKPO PUBLIC PHONE 2 LQ LEVEL 2


In [13]:
# drop Extension column
oct_2020.drop(['Extension'],axis= 1,inplace=True)

In [ ]:
# renaming Number to Extension
oct_2020.rename(columns = {'Number':'Extension'}, inplace = True)
list(oct_2020.columns)

In [ ]:
oct_2020.dtypes

In [ ]:
# view number of null columns
oct_2020.isnull().sum()

In [ ]:
oct_2020[oct_2020['Extension'].isnull()]

In [ ]:
# replace null values in extension
oct_2020['Extension'] = oct_2020['Extension'].fillna(0)

In [ ]:
# confirm changes
oct_2020.isnull().sum()

In [ ]:
# convert Extension column to appropriate datatype
convert_dict = {'Extension': int} 
  
oct_2020 = oct_2020.astype(convert_dict) 
oct_2020.dtypes

In [ ]:
# view source_data dataframe 
source_data.head(5)

In [ ]:
source_data.isnull().sum()

In [ ]:
#Rename telephoneNumber as Extension and Present Functional Name as Name to match oct_2020.
source_data.rename(columns = {'telephoneNumber':'Extension'}, inplace = True)
list(source_data.columns)

In [ ]:
source_data.dtypes

In [ ]:
merged_df = pd.merge(oct_2020, source_data, on ='Extension',how ='inner') 
merged_df.head()

In [ ]:
# drop unused columns
merged_df.drop(['Present Functional Name'],axis= 1,inplace=True)
# Change order of columns in dataframe
oct_2020_clean = merged_df[['Extension','Name','Division','Department','Duration','Calls','Cost']]

In [ ]:
oct_2020_clean.head()

In [ ]:
# check duplicated rows
oct_2020_clean.duplicated().sum()

In [ ]:
# drop duplicate rows using extension
oct_2020_clean = oct_2020_clean.drop_duplicates(subset='Extension', keep="first")

In [ ]:
# confirm changes
oct_2020_clean.duplicated().sum()

In [ ]:
oct_2020_clean.isnull().sum()

In [ ]:
# Output clean df
oct_2020_clean.to_excel("oct_2020_main_temp.xlsx",sheet_name='oct_main',index = False)

In [ ]:
# output null rows to dataframe with index
#null_values = oct_2020_clean[oct_2020_clean['Division'].isna()]
#null_values.to_excel("oct_2020_null.xlsx",sheet_name='null_values',index = False) 